In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}

url = 'http://www.pcbbbs.com/forum.php?gid=28'
r = requests.get(url, headers=headers).content.decode('gbk')

In [3]:
soup_string = BeautifulSoup(r, "html.parser")

In [4]:
table = soup_string.find('table',class_='fl_tb')

In [5]:
all_tr = table.findAll('tr')

In [6]:
all_tr[0]

<tr><td class="fl_icn" style="width: 72px;">
<a href="http://www.pcbbbs.com/forum.php?mod=forumdisplay&amp;fid=89"><img align="left" alt="" src="data/attachment/common/6d/common_31_icon.png"/></a></td>
<td>
<h2><a href="http://www.pcbbbs.com/forum.php?mod=forumdisplay&amp;fid=89">Altium Designer</a><em class="xw0 xi1" title="今日"> (1)</em></h2>
<p class="xg2">这个国内应用最广泛的设计用软件，成为进入PCB设计行业人员的首选。欢迎大家讨论使用心得。。</p><p>版主: <span class="xi2"><a c="1" class="notabs" href="http://home.pcbbbs.com/home.php?mod=space&amp;username=wanghanq">wanghanq</a>, <a c="1" class="notabs" href="http://home.pcbbbs.com/home.php?mod=space&amp;username=cai1116">cai1116</a>, <a c="1" class="notabs" href="http://home.pcbbbs.com/home.php?mod=space&amp;username=ziyu_00">ziyu_00</a>, <a c="1" class="notabs" href="http://home.pcbbbs.com/home.php?mod=space&amp;username=%BA%DA%C2%BF%CC%E3%D7%D3">黑驴蹄子</a></span></p></td>
<td class="fl_i">
<span class="xi2"><span title="12923">1万</span></span><span class="xg1"> / <span title="

In [7]:
all_tr[0].find('h2').find('a').attrs['href']

'http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=89'

In [8]:
href_1 = []

In [9]:
for item in all_tr:
    item_td = item.findAll('td')
    if len(item_td)>1:
        href = item_td[1].find('h2').find('a')
        if href.has_attr('href'):
            href_1.append([href.attrs['href'],href.text])

In [10]:
href_1

[['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=89',
  'Altium Designer'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=42', 'Allegro'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=45', 'Mentor'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=17', 'Pads'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=46', 'OrCAD'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=73', 'Zuken'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=111',
  'Cadence PCB SI'],
 ['http://www.pcbbbs.com/forum.php?mod=forumdisplay&fid=112', 'Hyperlynx']]

In [11]:
import os

In [12]:
def writefile(filename,content):
    with open(os.path.join('crawler',filename),'w+',encoding='utf-8') as f:
        f.write(content)

In [13]:
def tokenize(content):
    content = re.sub(r'[\n \r]+',r'',content)
    content = content.strip()
    return content

In [ ]:
for href in href_1:
    url_1 = href[0]
    print(url_1,href[1])
    r_1 = requests.get(url_1, headers=headers).content.decode('gbk')
    soup_string1 = BeautifulSoup(r_1, "html.parser")
    page_a = soup_string1.find('a',attrs={'class':'bm_h'})
    if page_a: total_page = int(page_a.attrs['totalpage'])
    else: total_page = 1
    for page in range(total_page):
        #if page>10: break
        print('page:{}'.format(page))
        url_page_1 = href[0]+'&page='+str(page+1)
        r_page_1 = requests.get(url_page_1, headers=headers).content.decode('gbk')
        soup_string_page_1 = BeautifulSoup(r_page_1, "html.parser")
        mn_div = soup_string_page_1.findAll('tbody',attrs={'id':re.compile(r"[stickthread | normalthread]_(\s\w+)?")})
        for href_2 in mn_div:
            url_2 = href_2.find('a').attrs['href']
            r_2 = requests.get(url_2, headers=headers).content.decode('gbk')
            soup_string2 = BeautifulSoup(r_2, "html.parser")
            content = soup_string2.find('td',attrs={'class':'t_f'})
            if content and content.text:
                content_text = tokenize(content.text)
                if content_text:
                    writefile(url_2.split('?')[1],content_text)
                    break